## Tensorflow中的正则化

### 参数加正则项
在损失函数上加上正则项是防止过拟合的一个重要方法,下面介绍如何在TensorFlow中使用正则项.

tensorflow中对参数使用正则项分为两步: 
- 创建一个正则方法(函数/对象) 
- 将这个正则方法(函数/对象),应用到参数上

#### 如何创建一个正则方法函数
```bash
tf.contrib.layers.l1_regularizer(scale, scope=None)
```
返回一个用来执行L1正则化的函数,函数的签名是func(weights). 

参数:

- scale: 正则项的系数.
- scope: 可选的scope name

```bash
tf.contrib.layers.l2_regularizer(scale, scope=None)
```

返回一个执行L2正则化的函数.
```bash
tf.contrib.layers.sum_regularizer(regularizer_list, scope=None)
```
返回一个可以执行多种(个)正则化的函数.意思是,创建一个正则化方法,这个方法是多个正则化方法的混合体.

参数: 
- regularizer_list: regulizer的列表

已经知道如何创建正则化方法了,下面要说明的就是如何将正则化方法应用到参数上

#### 应用正则化方法到参数上

```bash
tf.contrib.layers.apply_regularization(regularizer, weights_list=None)
```
参数:

- regularizer:就是我们上一步创建的正则化方法
- weights_list: 想要执行正则化方法的参数列表,如果为None的话,就取GraphKeys.WEIGHTS中的weights.

函数返回一个标量Tensor,同时,这个标量Tensor也会保存到GraphKeys.REGULARIZATION_LOSSES中.这个Tensor保存了计算正则项损失的方法.

- tensorflow中的Tensor是保存了计算这个值的路径(方法),当我们run的时候,tensorflow后端就通过路径计算出Tensor对应的值。

现在,我们只需将这个正则项损失加到我们的损失函数上就可以了.

- 如果是自己手动定义weight的话,需要手动将weight保存到GraphKeys.WEIGHTS中,但是如果使用layer的话,就不用这么麻烦了,别人已经帮你考虑好了.(最好自己验证一下tf.GraphKeys.WEIGHTS中是否包含了所有的weights,防止被坑)

### 其他方法

- 在使用tf.get_variable()和tf.variable_scope()的时候,你会发现,它俩中有regularizer形参.如果传入这个参数的话,那么variable_scope内的weights的正则化损失,或者weights的正则化损失就会被添加到GraphKeys.REGULARIZATION_LOSSES中

In [1]:
import tensorflow as tf
from tensorflow.contrib import layers

# 在tf.variable_scope()中加入regulizer
regularizer = layers.l1_regularizer(0.1)
with tf.variable_scope('var', initializer=tf.random_normal_initializer(), 
regularizer=regularizer):
    weight = tf.get_variable('weight', shape=[8], initializer=tf.ones_initializer())
    
with tf.variable_scope('var2', initializer=tf.random_normal_initializer(), 
regularizer=regularizer):
    weight2 = tf.get_variable('weight', shape=[8], initializer=tf.ones_initializer())

regularization_loss = tf.reduce_sum(tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES))

### 网络层加Dropout
- tf.nn.dropout(x, keep_prob, noise_shape=None, seed=None, name=None)
- tf.nn.rnn_cell.DropoutWrapper(rnn_cell, input_keep_prob=1.0, output_keep_prob=1.0)

```bash
tf.nn.dropout(x, keep_prob, noise_shape=None, seed=None, name=None)
```
- 第一个参数x：指输入

- 第二个参数keep_prob: 设置神经元被选中的概率,在初始化时keep_prob是一个占位符,  keep_prob = tf.placeholder(tf.float32) 。tensorflow在run时设置keep_prob具体的值，例如keep_prob: 0.5


In [ ]:
# 普通的dropout
def dropout(x, keep_prob, noise_shape=None, seed=None, name=None)
    w = tf.get_variable("w1",shape=[size, out_size])
    x = tf.placeholder(tf.float32, shape=[batch_size, size])
    x = tf.nn.dropout(x, keep_prob=0.5)
    y = tf.matmul(x,w)

In [ ]:
# rnn中的dropout
def rnn_cell.DropoutWrapper(rnn_cell, input_keep_prob=1.0, output_keep_prob=1.0):
    lstm_cell = tf.nn.rnn_cell.BasicLSTMCell(size, forget_bias=0.0, state_is_tuple=True)
    lstm_cell = tf.nn.rnn_cell.DropoutWrapper(lstm_cell, output_keep_prob=0.5)
